In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np

pybullet build time: Nov 28 2023 23:45:17


In [2]:
import xml.etree.ElementTree as ET

def parse_collision_boxes(urdf_file):
    # Parse the URDF file
    tree = ET.parse(urdf_file)
    root = tree.getroot()

    boxes = []
    
    for link in root.findall("link"):
        for collision in link.findall("collision"):
            geometry = collision.find("geometry")
            box = geometry.find("box")
            if box is not None:
                size = list(map(float, box.attrib["size"].split()))
                origin_elem = collision.find("origin")
                if origin_elem is not None:
                    origin = list(map(float, origin_elem.attrib["xyz"].split()))
                    rpy = list(map(float, origin_elem.attrib["rpy"].split()))
                else:
                    origin = [0.0, 0.0, 0.0]
                    rpy = [0.0, 0.0, 0.0]
                boxes.append((size, origin, rpy))
    return boxes

def compute_min_max(size, origin, rpy):
    # Unpack size and origin
    sx, sy, sz = size
    ox, oy, oz = origin
    
    # Half extents
    hx, hy, hz = sx / 2, sy / 2, sz / 2
    
    # Define corners of the box relative to origin
    corners = np.array([
        [-hx, -hy, -hz],
        [-hx, -hy, hz],
        [-hx, hy, -hz],
        [-hx, hy, hz],
        [hx, -hy, -hz],
        [hx, -hy, hz],
        [hx, hy, -hz],
        [hx, hy, hz],
    ])
    
    # Rotation matrix for z-axis only (rpy[2])
    theta = rpy[2]
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta),  np.cos(theta), 0],
        [0, 0, 1]
    ])
    
    # Rotate and translate corners
    transformed_corners = np.dot(corners, rotation_matrix.T) + np.array([ox, oy, oz])
    
    # Get min and max
    min_coords = transformed_corners.min(axis=0)
    max_coords = transformed_corners.max(axis=0)
    
    return min_coords, max_coords

In [3]:
urdf_file = "model.urdf"
boxes = parse_collision_boxes(urdf_file)

global_min = np.array([float('inf'), float('inf'), float('inf')])
global_max = np.array([-float('inf'), -float('inf'), -float('inf')])

mins = []
maxs = []
for size, origin, rpy in boxes:
    min_coords, max_coords = compute_min_max(size, origin, rpy)
    mins.append(min_coords)
    maxs.append(max_coords)
    #print(f"Box: Size={size}, Origin={origin}, RPY={rpy}")
    #print(f"  Min: {min_coords}")
    #print(f"  Max: {max_coords}")
    
    # Update global bounds
    global_min = np.minimum(global_min, min_coords)
    global_max = np.maximum(global_max, max_coords)

print("\nGlobal Bounds:")
print(f"  Global Min: {global_min.round(3)}")
print(f"  Global Max: {global_max.round(3)}")


Global Bounds:
  Global Min: [-2.275 -2.525  0.   ]
  Global Max: [2.275 2.025 0.3  ]


In [4]:
buffer=0.1
adjusted_min = np.array(mins) - buffer
adjusted_max = np.array(maxs) + buffer

In [5]:
def check_collision(coords):
    coords = np.array(coords)
    if coords.shape[0] == 2:
        coords = np.array(list(coords) + [0.1])
    for i in range(len(boxes)):
        if np.all((coords >= adjusted_min[i]) & (coords <= adjusted_max[i])):
            return True

    return False

In [6]:
def update_sphere_color(sphere_id, sphere_position):
    # Condition: Change color if the sphere's x position is greater than 1
    if check_collision(sphere_position):
        # Set color to red
        p.changeVisualShape(sphere_id, -1, rgbaColor=[1, 0, 0, 1])
    else:
        # Set color to green
        p.changeVisualShape(sphere_id, -1, rgbaColor=[0, 1, 0, 1])

In [7]:
def euclidean_distance(point1, point2):
    point1 = np.array(point1)  # Convert to numpy array if not already
    point2 = np.array(point2)  # Convert to numpy array if not already
    return np.linalg.norm(point1 - point2)

In [8]:
threshold = 0.01

In [9]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Function to interpolate between two points
def interpolate_2d(q, closest_neighbour, step=0.05):
    dist = np.linalg.norm(q - closest_neighbour)
    steps = int(dist / step)
    return np.linspace(q, closest_neighbour, steps+1)


In [10]:
def RRT_build(q_0, q_goal, n=5000):
    V = [q_0]  # List of vertices (nodes)
    E = []  # List of edges (connections between nodes)
    bounds = [-2.275, 2.275, -2.525, 2.025]  # Bounds for random sampling
    goal_reached = False
    count = 0
    
    for _ in range(n):
        count = count + 1
        print('New sample:', count)
        if goal_reached:  # If the goal is reached, exit the loop
            break
        
        q = np.array([random.uniform(bounds[0], bounds[1]), random.uniform(bounds[2], bounds[3])])

        # Keep sampling new points if they collide with an obstacle
        while check_collision(q):
            q = np.array([random.uniform(bounds[0], bounds[1]), random.uniform(bounds[2], bounds[3])])
            
            
        
        # Find the closest existing node in the tree
        distances = [np.linalg.norm(q - v) for v in V]
        closest_neighbour = V[np.argmin(distances)]

        # Interpolate between the new point and the closest neighbour
        interpolated = interpolate_2d(q, closest_neighbour)
        collision_free = all(not check_collision(point) for point in interpolated)

        # If the path is collision-free, add the point and edge to the tree
        if collision_free:
            V.append(q)
            E.append([q, closest_neighbour])
            print('collision free')

            # Check if the path to the goal is collision-free
            interpolated_goal = interpolate_2d(q, q_goal)
            collision_free_goal = all(not check_collision(point) for point in interpolated_goal)

            # If the path to the goal is also collision-free, add the goal and edge, then stop
            if collision_free_goal:
                print('path to goal good')
                E.append([q, q_goal])
                V.append(q_goal)
                goal_reached = True
                # break
    return V, E

# Function to reconstruct the path from the tree
def find_RRT_path(E, q_0, q_goal):
    #print(E, q_0, q_goal)
    path = [q_goal, E[-1][0]]  # Start path with the goal
    while not np.allclose(path[-1], q_0, atol=1e-6):  # Loop until the start is reached
        for edge in E:
            if np.allclose(edge[0], path[-1], atol=1e-6):  # Find the edge leading to the last point in the path
                path.append(edge[1])  # Add the next point in the path
                break
    path.reverse()  # Reverse the path to go from start to goal
    return path


def rrt_final(q_0, q_goal, n=5000):
    print("Initial:", q_0[:2], "Goal: ", q_goal[:2])
    V, E = RRT_build(q_0[:2], q_goal[:2], n)
    #print("V: ", V, "E: ", E)
    path = find_RRT_path(E, q_0[:2], q_goal[:2])
    print(path)
    return path

In [11]:
def follower(robot_id, target_position, speed):
    robot_position, robot_orientation = p.getBasePositionAndOrientation(robot_id)
    robot_position = np.array(robot_position)
    target_position = np.array(target_position)
    path = rrt_final(robot_position, target_position, n=5000)
    for node in path:
        node = np.array(list(node) + [0.1])
        target_position = node
        while euclidean_distance(robot_position[:2], target_position[:2]) > threshold:
            difference = target_position - robot_position
            difference[2] = 0
            robot_position = robot_position + (speed/ np.linalg.norm(difference))*difference
            p.resetBasePositionAndOrientation(robot_id, list(robot_position), robot_orientation)
            p.stepSimulation()
            time.sleep(1. / 240.)

In [12]:
import pybullet as p
import pybullet_data
import time

# Connect to PyBullet with GUI
physicsClient = p.connect(p.GUI)

# Load environment
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.8)
p.loadURDF("plane.urdf")  # Ground plane

urdf_path = "model.urdf"
robot_id = p.loadURDF(urdf_path, basePosition=[0, 0, 0.5])  # Adjust the position as needed

# Set starting position and orientation for the drone
#startPos = [0, -0.5, 2]
startPos = [0, -0.5, 0.1]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
# Load a sample drone (use a valid URDF file here)
# This scales the robot by adjusting the position for scaling (approximates rescaling)
#droneId = p.loadURDF("r2d2.urdf", startPos, startOrientation, globalScaling=0.5)


# Add a sphere to the simulation
sphere_radius = 0.1
sphere_visual_shape = p.GEOM_SPHERE
#sphere_collision_shape = p.createCollisionShape(sphere_visual_shape, radius=sphere_radius)
sphere_visual = p.createVisualShape(sphere_visual_shape, radius=sphere_radius, rgbaColor=[1, 0, 0, 1])

sphere_start_position = [0, 0, 0.1]
# sphere_id = p.createMultiBody(baseMass=0,  # Static object
#                               baseCollisionShapeIndex=sphere_collision_shape,
#                               baseVisualShapeIndex=sphere_visual,
#                               basePosition=sphere_start_position)
sphere_id = p.createMultiBody(baseMass=0,  # Static object
                              baseVisualShapeIndex=sphere_visual,
                              basePosition=sphere_start_position)
droneId = p.createMultiBody(baseMass=0,  # Static object
                              baseVisualShapeIndex=sphere_visual,
                              basePosition=startPos)
# Adjust the camera to focus on the drone
p.resetDebugVisualizerCamera(cameraDistance=3, cameraYaw=50, cameraPitch=-30, cameraTargetPosition=startPos)

# Sphere movement variables
sphere_speed = 0.01
sphere_position = list(sphere_start_position)

# Run simulation
while True:
    keys = p.getKeyboardEvents()
    if ord('q') in keys and keys[ord('q')] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
        break
    # Move sphere continuously based on key input
    if p.B3G_UP_ARROW in keys and keys[p.B3G_UP_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
        sphere_position[1] += sphere_speed
    if p.B3G_DOWN_ARROW in keys and keys[p.B3G_DOWN_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
        sphere_position[1] -= sphere_speed
    if p.B3G_LEFT_ARROW in keys and keys[p.B3G_LEFT_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
        sphere_position[0] -= sphere_speed
    if p.B3G_RIGHT_ARROW in keys and keys[p.B3G_RIGHT_ARROW] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
        sphere_position[0] += sphere_speed
    if ord('f') in keys and keys[ord('f')] & (p.KEY_IS_DOWN | p.KEY_WAS_TRIGGERED):
        p.changeVisualShape(sphere_id, -1, rgbaColor=[0, 0, 1, 1])    

        follower(droneId, sphere_position, sphere_speed)

    # Update sphere position
    p.resetBasePositionAndOrientation(sphere_id, sphere_position, [0, 0, 0, 1])

    update_sphere_color(sphere_id, sphere_position)

    # Step simulation
    p.stepSimulation()
    time.sleep(1. / 240.)  # Slow simulation for visualization

# Disconnect
p.disconnect()


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 550.120
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 550.120
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial dat